# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

import os

pd.set_option('display.max_colwidth', None)

movies_path="data/movies"
print(os.listdir(movies_path))

['keywords.csv', 'credits.csv', 'ratings.csv', 'movies_metadata.csv', 'links.csv', 'links_small.csv', 'ratings_small.csv']


#### Common functions

In [2]:
def info(df):
    display(df.info())
    
    print("Null values count:")
    print()
    c=df.isnull().sum()
    print(c[c>0])
    
    print()
    print("Statistics:")
    display(df.describe(include='all'))
    
    print("Head:")
    display(df.head())

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

Explain what end use cases you'd like to prepare the data for (e.g., analytics table, app back-end, source-of-truth database, etc.)

The goal of this project is to prepare datawarehouse source of truth/analytics table to be able to:
1) Create user recomendations of movies based on popularity/rating

2) TODO:

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 


The MovieLens dataset comes from [this kaggle page](https://www.kaggle.com/rounakbanik/the-movies-dataset)

This dataset consists of the following files:

**movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

**keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

**credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

**links.csv**: The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.

**links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

**ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

The Full MovieLens Dataset consisting of 26 million ratings and 750,000 tag applications from 270,000 users on all the 45,000 movies in this dataset can be accessed here

In [3]:
#Create spark session
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

21/11/21 18:38:48 WARN Utils: Your hostname, babkamen-Lenovo resolves to a loopback address: 127.0.1.1; using 192.168.0.133 instead (on interface wlp3s0)
21/11/21 18:38:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/21 18:38:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#Read data

credits_df=spark.read.csv(f"{movies_path}/credits.csv", inferSchema=True, header=True)

keywords_df=spark.read.csv(f"{movies_path}/keywords.csv", inferSchema=True, header=True)

links_df=spark.read.csv(f"{movies_path}/links.csv", inferSchema=True, header=True)

movies_metadata_df=spark.read.csv(f"{movies_path}/movies_metadata.csv", inferSchema=True, header=True)

ratings_df=spark.read.csv(f"{movies_path}/ratings_small.csv", inferSchema=True, header=True)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

#### Credits


In [5]:
credits=credits_df.toPandas()

In [6]:
info(credits)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45475 non-null  object
 2   id      45476 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


None

Null values count:

crew    1
dtype: int64

Statistics:


,cast,crew,id
count,45476,45475,45476
unique,42987,41476,31677
top,[],[],'gender': 2
freq,2418,742,4113


Head:


cast  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'}, {'cast_id': 19, 'character': 'Hamm (voice)', 'credit_id': '52fe4284c3a36847f8024fa9', 'gender': 2, 'id': 7907, 'name': 'John Ratzenberger', 'order': 5, 'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'}, {'cast_id': 20, 'character': 'Bo Peep (voice)', 'credit_id': '52fe4284c3a36847f8024fad', 'gender': 1, 'id': 8873, 'name': 'Annie Potts', 'order': 6, 'profile_path': '/eryXT84RL41jHSJcMy4kS3u9y6w.jpg'}, {'cast_id': 26, 'character': 'Andy (voice)', 'credit_id': '52fe4284c3a36847f8024fc1', 'gender': 0, 'id': 1116442, 'name': 'John Morris', 'order': 7, 'profile_path': '/vYG

# TODO: 
1) handle parsing of the crew correctly so that all ids are numbers
2) There is None in the json(cast column )
3) 

In [14]:
print("Ids that contain alphabetic symbols:")
credits.id.str.contains("[A-Za-z]",regex=True).sum()

Ids that contain alphabetic symbols:


15019

Schema: 
- cast: json array
    - cast_id: number
    - character: string
    - credit_id: string
    - gender: number
    - id: number
    - name: string
    - order: number
    - profile_path: url path
- crew: json array
    - credit_id: string(uuid?)
    - department: string
    - gender: number
    - id: number
    - job: string
    - name: string
    - profile: url path
- id: number


Let's look closer into columns structure:

cast:
```json
[
   {
      "cast_id":14,
      "character":"Woody (voice)",
      "credit_id":"52fe4284c3a36847f8024f95",
      "gender":2,
      "id":31,
      "name":"Tom Hanks",
      "order":0,
      "profile_path":"/pQFoyx7rp09CJTAb932F2g8Nlho.jpg"
   },
   ...
```

crew:

```json
[
   {
      "credit_id":"52fe44bfc3a36847f80a7cd1",
      "department":"Production",
      "gender":2,
      "id":511,
      "job":"Executive Producer",
      "name":"Larry J. Franco",
      "profile_path":"None"
   },
   ...
```

#### Keywords

In [8]:
info(keywords_df.toPandas())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int32 
 1   keywords  46419 non-null  object
dtypes: int32(1), object(1)
memory usage: 544.1+ KB


None

Null values count:

Series([], dtype: int64)

Statistics:


,id,keywords
count,46419.000000,46419
unique,NaN,25951
top,NaN,[]
freq,NaN,14795
mean,109769.951873,NaN
std,113045.780256,NaN
min,2.000000,NaN
25%,26810.500000,NaN
50%,61198.000000,NaN
75%,159908.500000,NaN


Head:


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"
1,8844,"""[{'id': 10090, 'name': 'board game'}, {'id': 10941, 'name': 'disappearance'}, {'id': 15101, 'name': """"based on children's book""""}"
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392, 'name': 'best friend'}, {'id': 179431, 'name': 'duringcreditsstinger'}, {'id': 208510, 'name': 'old men'}]"
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id': 10131, 'name': 'interracial relationship'}, {'id': 14768, 'name': 'single mother'}, {'id': 15160, 'name': 'divorce'}, {'id': 33455, 'name': 'chick flick'}]"
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'name': 'midlife crisis'}, {'id': 2246, 'name': 'confidence'}, {'id': 4995, 'name': 'aging'}, {'id': 5600, 'name': 'daughter'}, {'id': 10707, 'name': 'mother daughter relationship'}, {'id': 13149, 'name': 'pregnancy'}, {'id': 33358, 'name': 'contraception'}, {'id': 170521, 'name': 'gynecologist'}]"


Schema:

- id - TODO: check what id is that
- keywords: json
    - id: number
    - name: string

#### Links

In [9]:
info(links_df.toPandas())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int32  
 1   imdbId   45843 non-null  int32  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int32(2)
memory usage: 716.4 KB


None

Null values count:

tmdbId    219
dtype: int64

Statistics:


,movieId,imdbId,tmdbId
count,45843.000000,4.584300e+04,45624.000000
mean,96578.775626,9.937080e+05,108661.382847
std,57216.863469,1.361924e+06,112665.970830
min,1.000000,1.000000e+00,2.000000
25%,49202.500000,8.333050e+04,26502.750000
50%,108799.000000,2.839910e+05,60178.000000
75%,145270.500000,1.538311e+06,157849.500000
max,176279.000000,7.158814e+06,469172.000000


Head:


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


Schema:
- movieId: number
- imbid: number
- tmdbid: number

# TODO: convert tmdbid to int

#### Movies metadata

In [10]:
info(movies_metadata_df.toPandas())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45572 entries, 0 to 45571
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   adult                  45572 non-null  object
 1   belongs_to_collection  4591 non-null   object
 2   budget                 45555 non-null  object
 3   genres                 45549 non-null  object
 4   homepage               7937 non-null   object
 5   id                     45541 non-null  object
 6   imdb_id                45447 non-null  object
 7   original_language      45527 non-null  object
 8   original_title         45540 non-null  object
 9   overview               44587 non-null  object
 10  popularity             45452 non-null  object
 11  poster_path            45091 non-null  object
 12  production_companies   45448 non-null  object
 13  production_countries   45452 non-null  object
 14  release_date           45378 non-null  object
 15  revenue            

None

Null values count:

belongs_to_collection    40981
budget                      17
genres                      23
homepage                 37635
id                          31
imdb_id                    125
original_language           45
original_title              32
overview                   985
popularity                 120
poster_path                481
production_companies       124
production_countries       120
release_date               194
revenue                    136
runtime                    404
spoken_languages           164
status                     245
tagline                  23132
title                      778
video                      553
vote_average               498
vote_count                 389
dtype: int64

Statistics:


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
count,45572,4591,45555,45549,7937,45541,45447,45527,45540,44587,...,45378,45436,45168,45408,45327,22440,44794,45019,45074,45183
unique,111,1795,1358,4172,7740,45469,45386,246,43418,44253,...,19065,9169,2535,3613,1367,19588,40102,2102,1573,2898
top,False,"{'id': 415931, 'name': 'The Bowery Boys', 'poster_path': '/q6sA4bzMT9cK7EEmXYwt7PNrL5h.jpg', 'backdrop_path': '/foe3kuiJmg5AklhtD3skWbaTMf2.jpg'}",0,"[{'id': 18, 'name': 'Drama'}]",0,"[{'id': 35, 'name': 'Comedy'}]",0,en,0,No overview found.,...,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Released,Released,False,0.0,1
freq,45454,29,36509,4996,61,15,20,32185,16,133,...,478,34799,2334,20523,41194,1090,749,41496,2756,2972


Head:


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,False,7.7,5415
1,False,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",None,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",None,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92
3,False,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",None,31357,tt0114885,en,Waiting to Exhale,"""Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive """"good man"""" to break a string of less-than-stellar lovers. Friends and confidants Vannah",...,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",None,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.",...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173


Schema: 
- adult: boolean. Descr - adult content?
- belongs_to_collection: json element
    - id: number
    - name: string
    - poster: url path
    - backdrop_path: url path
- budget: number
- genres: json array
    - id: number
    - name: string
- homepage: string
- id: number
- imdb_id: string. Always starts with tt*
- original_language: string
- original_title: string
- overview: string
- popularity: TODO
- poster_path: TODO
- production_companies: TODO
- production_countries: TODO
- release_date: date (YYYY-MM-dd)
- revenue: number 
- runtime: float. ??
- spoken_languages: json array
    - iso_639_1: string
    - name: string
- status: string. TODO: check if there is enum for that
- tagline: string
- title: string
- video: boolean
- vote_average: decimal
- vote_count: number


#### Ratings

In [11]:
info(ratings_df.toPandas())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int32  
 1   movieId    100004 non-null  int32  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int32  
dtypes: float64(1), int32(3)
memory usage: 1.9 MB


None

Null values count:

Series([], dtype: int64)

Statistics:


,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


Head:


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Schema: 
- userId: number
- movieId: number
- rating: decimal
- timestamp: number

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [12]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [13]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.